In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt5
# Add modules to the path
import sys
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git')
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos')
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git\\freely_moving_photometry_analysis' )

import numpy as np
from sklearn.linear_model import LinearRegression
from scipy.ndimage.interpolation import shift
from scipy.signal import decimate
from matplotlib import pyplot as plt
from scipy import stats
from scipy.stats import sem
import pandas as pd
import pickle
from matplotlib import colors, cm
from linear_regression.get_time_stamps_for_regression import remove_manipulation_days



In C:\Users\francescag\AppData\Local\Continuum\miniconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In C:\Users\francescag\AppData\Local\Continuum\miniconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In C:\Users\francescag\AppData\Local\Continuum\miniconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


In [2]:
experiment_record = pd.read_csv('W:\\photometry_2AC\\experimental_record.csv')
experiment_record['date'] = experiment_record['date'].astype(str)
clean_experiments = remove_manipulation_days(experiment_record)

In [3]:
param_names = ['ipsi choices', 'contra choices', 'cues', 'rewards']

In [8]:
ipsi_choice_kernel = []


In [38]:
mouse_ids = [ 'SNL_photo17']

In [39]:
experiments_to_process = clean_experiments[(clean_experiments['mouse_id'].isin(mouse_ids) ) & (clean_experiments['recording_site'] == 'tail')].reset_index(drop=True)

In [40]:
experiments_to_process

,mouse_id,date,recording_site,fiber_side,experiment_notes,recording_notes,Unnamed: 6,include
0,SNL_photo17,20200204,tail,right,NaN,NaN,NaN,NaN
1,SNL_photo17,20200206,tail,right,NaN,NaN,NaN,NaN
2,SNL_photo17,20200208,tail,right,NaN,NaN,NaN,NaN
3,SNL_photo17,20200210,tail,right,NaN,NaN,NaN,NaN
4,SNL_photo17,20200212,tail,right,NaN,NaN,NaN,NaN
5,SNL_photo17,20200214,tail,right,NaN,NaN,NaN,NaN
6,SNL_photo17,20200218,tail,right,NaN,NaN,NaN,NaN
7,SNL_photo17,20200221,tail,right,NaN,NaN,NaN,NaN
8,SNL_photo17,20200224,tail,right,NaN,NaN,NaN,NaN
9,SNL_photo17,20200226,tail,right,NaN,NaN,NaN,NaN


In [41]:
for index, experiment in experiments_to_process.iterrows():
    mouse = experiment['mouse_id']
    date = experiment['date']
    saving_folder = 'W:\\photometry_2AC\\processed_data\\' + mouse + '\\'
    save_filename = mouse + '_' + date + '_' + 'linear_regresion_kernels.p'
    with open(saving_folder + save_filename, "rb") as f:
        session_kernels = pickle.load(f)
        if index == 0:
            ipsi_choice_kernel = session_kernels['kernels']['ipsi choices']
            contra_choice_kernel = session_kernels['kernels']['contra choices']
            cue_kernel = session_kernels['kernels']['cues']
            reward_kernel = session_kernels['kernels']['rewards']
            time_stamps = session_kernels['time_stamps']
        else:
            ipsi_choice_kernel = np.vstack([ipsi_choice_kernel, session_kernels['kernels']['ipsi choices']])
            contra_choice_kernel = np.vstack([contra_choice_kernel, session_kernels['kernels']['contra choices']])
            cue_kernel = np.vstack([cue_kernel, session_kernels['kernels']['cues']])
            reward_kernel = np.vstack([reward_kernel, session_kernels['kernels']['rewards']])



In [42]:
mean_kernels = {}
mean_kernels['ipsi choices'] = np.mean(ipsi_choice_kernel, axis=0)
mean_kernels['contra choices'] = np.mean(contra_choice_kernel, axis=0)
mean_kernels['cues'] = np.mean(cue_kernel, axis=0)
mean_kernels['rewards'] = np.mean(reward_kernel, axis=0)

In [43]:
sem_kernels = {}
sem_kernels['ipsi choices'] = sem(ipsi_choice_kernel, axis=0)
sem_kernels['contra choices'] = sem(contra_choice_kernel, axis=0)
sem_kernels['cues'] = sem(cue_kernel, axis=0)
sem_kernels['rewards'] = sem(reward_kernel, axis=0)

In [44]:
fig, axs = plt.subplots(nrows=1, ncols=len(param_names), sharey=True, figsize=(11,6))
axs[0].set_ylabel('Regression coefficient', fontsize=12)
for param_num, param_name in enumerate(param_names):
    param_kernel = mean_kernels[param_name]
    axs[param_num].plot(time_stamps, param_kernel, label=param_name)
    axs[param_num].axvline(0, color='k')
    axs[param_num].fill_between(time_stamps, param_kernel - sem_kernels[param_name] ,  param_kernel + sem_kernels[param_name], alpha=0.5,
                            facecolor='#7FB5B5', linewidth=0)
    axs[param_num].set_title(param_name)
    axs[param_num].set_xlabel('Time (s)', fontsize=12)
plt.tight_layout()

In [15]:
plot_filename = 'SNL_photo16_17_18_21_22_26_mean_regression_kernels.pdf' 
plot_save_folder = 'W:\\photometry_2AC\\plots\\Linear regression\\'
plt.savefig(plot_save_folder + plot_filename, transparent=True, optimize=True)

In [100]:
experiments_record_tail = experiment_record[experiment_record['recording_site'] == 'tail']

,mouse_id,date,recording_site,fiber_side
28,SNL_photo18,20200223,tail,left
30,SNL_photo18,20200226,tail,left
32,SNL_photo18,20200228,tail,left
33,SNL_photo18,20200229,tail,left
35,SNL_photo18,20200303,tail,left
37,SNL_photo18,20200305,tail,left
39,SNL_photo18,20200307,tail,left
40,SNL_photo18,20200310,tail,left
42,SNL_photo18,20200312,tail,left
43,SNL_photo18,20200313,tail,left


In [99]:
for mouse_id in experiment_record_tail['mouse_id'].unique():
    experiments = experiment_record_tail[experiment_record_tail['mouse_id'] == mouse_id]
    for index, experiment in experiments.iterrows():
        mouse = experiment['mouse_id']
        date = experiment['date']
        saving_folder = 'W:\\photometry_2AC\\processed_data\\' + mouse + '\\'
        save_filename = mouse + '_' + date + '_' + 'linear_regresion_kernels.p'
        with open(saving_folder + save_filename, "rb") as f:
            session_kernels = pickle.load(f)
            if index == 1:
                ipsi_choice_kernel = session_kernels['kernels']['ipsi choices']
                contra_choice_kernel = session_kernels['kernels']['contra choices']
                cue_kernel = session_kernels['kernels']['cues']
                reward_kernel = session_kernels['kernels']['rewards']
                time_stamps = session_kernels['time_stamps']
            else:
                ipsi_choice_kernel = np.vstack([ipsi_choice_kernel, session_kernels['kernels']['ipsi choices']])
                contra_choice_kernel = np.vstack([contra_choice_kernel, session_kernels['kernels']['contra choices']])
                cue_kernel = np.vstack([cue_kernel, session_kernels['kernels']['cues']])
                reward_kernel = np.vstack([reward_kernel, session_kernels['kernels']['rewards']])

    all_kernels={}
    all_kernels['ipsi choices'] = ipsi_choice_kernel
    all_kernels['contra choices'] = contra_choice_kernel
    all_kernels['cues'] = cue_kernel
    all_kernels['rewards'] = reward_kernel
    
    mean_kernels = {}
    mean_kernels['ipsi choices'] = np.mean(ipsi_choice_kernel, axis=0)
    mean_kernels['contra choices'] = np.mean(contra_choice_kernel, axis=0)
    mean_kernels['cues'] = np.mean(cue_kernel, axis=0)
    mean_kernels['rewards'] = np.mean(reward_kernel, axis=0)
    
    std_kernels = {}
    std_kernels['ipsi choices'] = np.std(ipsi_choice_kernel, axis=0)
    std_kernels['contra choices'] = np.std(contra_choice_kernel, axis=0)
    std_kernels['cues'] = np.std(cue_kernel, axis=0)
    std_kernels['rewards'] = np.std(reward_kernel, axis=0)

    fig, axs = plt.subplots(nrows=1, ncols=len(param_names), sharey=True, figsize=(15,8))
    axs[0].set_ylabel('Regression coefficient')
    for param_num, param_name in enumerate(param_names):
        param_kernel = all_kernels[param_name]
        axs[param_num].set_prop_cycle(plt.cycler('color',colours))
        axs[param_num].plot(time_stamps,param_kernel.T)

        axs[param_num].set_title(param_name)
        axs[param_num].set_xlabel('Time (s)')
        axs[param_num].axvline(0.2)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
30
32
33
35
37
39
40
42
43
44


In [94]:
first_bin = np.where(time_stamps == 0.2)
np.squeeze(contra_choice_kernel[:,first_bin])

array([0.87801853, 0.61416135, 1.10047466, 0.68212124, 1.1398374 ,
       0.82871293, 1.29255329, 0.77321524, 0.87099428, 0.92496414,
       0.65088095, 1.45326512, 1.19129577, 1.05429092, 0.95228183,
       0.40851538, 0.80581603, 0.6517777 , 0.15186452, 0.7016423 ,
       0.54395115, 0.51660555, 0.18614848, 0.37883117, 0.04421242,
       0.21309765, 0.49045478, 1.78318808, 0.98105266, 0.748314  ,
       0.5835751 , 0.86904679, 0.8399813 , 0.63625214, 0.7217782 ,
       0.66097678, 0.60119746, 0.58549881])

In [95]:
plt.plot(np.squeeze(contra_choice_kernel[:,first_bin]))

In [96]:
experiments

,mouse_id,date,recording_site,fiber_side
28,SNL_photo18,20200223,tail,left
30,SNL_photo18,20200226,tail,left
32,SNL_photo18,20200228,tail,left
33,SNL_photo18,20200229,tail,left
35,SNL_photo18,20200303,tail,left
37,SNL_photo18,20200305,tail,left
39,SNL_photo18,20200307,tail,left
40,SNL_photo18,20200310,tail,left
42,SNL_photo18,20200312,tail,left
43,SNL_photo18,20200313,tail,left


In [97]:
contra_choice_kernel.shape

(38, 301)